In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import requests

In [ ]:
crude_oil_ticker = "CL=F"
exr_eurusd_ticker = "EURUSD=X"
hist = crude_oil.history()

In [ ]:
data = yf.download(crude_oil_ticker)
exr = yf.download(exr_eurusd_ticker)

In [ ]:
# Затраты на производство
PRODUCTION_COST = 400 # (EUR)

# Расходы на логистику
EU_LOGISTIC_COST_EUR = 30 # в Европу в евро
CN_LOGISTIC_COST_USD = 130 # в Китай в долларах

# * Справочная информация по клиентам(объемы, локации, комментарии) 
customers = {
    'Сonty':{
        'location':'EU',
        'volumes':200,
        'comment':'moving_average'
    },
    
    'Triangle':{
        'location':'CN',
        'volumes': 30,
        'comment': 'monthly'
    },
    'Stone':{
        'location':'EU',
        'volumes': 150,
        'comment': 'moving_average'
    },
    'Ant':{
        'location':'EU',
        'volumes': 70,
        'comment': 'monthly'
    }
}
# Скидки
discounts = {'99': 0.01, # до 100 тонн 1%
             '299': 0.05, #  до 300 тонн 5%
             '300': 0.1}   # больше 300 тонн 10%


In [ ]:
year = '2023'
month = '03'

grade = url = f"https://www.lgm.gov.my/webv2api/api/rubberprice/month={month}&year={year}"
res = requests.get(url)
rj = res.json()
df2 = pd.json_normalize(rj)
df2.head(10)

In [ ]:
df2['us'] = pd.to_numeric(df2['us'], errors='coerce')
df2.dropna(inplace=True)
df2['trunc'] = pd.to_datetime(df2['date']).dt.to_period('M')
df2 = df2[df2['grade']=='SMR 10']
result = df2.groupby('trunc')['us'].mean()
df_now = result.to_frame(name='average_us') * 10

In [ ]:
df = pd.concat([data.resample('M').mean()['Close'], exr.resample('M').mean()['Close']], axis=1)
df.columns = ['crude_oil_price', 'eureusd']
df = df['2022-01-01':'2022-12-31']
# # Formula (10*Data + 400 ) * Discount + Logistics
df["proccessing_usd"] = df['eureusd'] * 400
df["base_wpb_price_usd"] = df["crude_oil_price"]*10 + df['proccessing_usd']

In [ ]:
def check_volume_discounts(x):
  if 0 < x <= 99:
    return 0.01
  elif 99 < x <= 299:
    return 0.05
  elif x >= 300:
    return 0.1
  else:
    return 0

In [ ]:
def price_now(x,y,z,w):
    new = x * (1-y) + z
    if new > w:
        return new
    else: 
        return w

In [1]:
clients_path = 'for_clients'
os.makedirs(clients_path, exist_ok=True)

for client_name, client_info in customers.items():
    client_df = df.copy()
    
    if client_info.get('location') == 'EU':
      client_df['logistics'] = client_df['eureusd'] * EU_LOGISTIC_COST_EUR
    elif client_info.get('location') == 'CN':
      client_df['logistics'] = CN_LOGISTIC_COST_USD
    
    if client_info.get('volumes'):
      client_df['volumes'] = client_info.get('volumes')

    client_df['discount'] = client_df['volumes'].apply(check_volume_discounts)
    average_us = df_now['average_us'][0]
    client_df['WBP_price_usd'] = client_df.apply(lambda x: price_now(x['base_wpb_price_usd'],x['discount'],x['logistics'],average_us), axis=1)
    
    client_df['WBP_price_usd'].plot(color='red', linestyle="dashed")
    plt.title(f"Client {client_name} WBP Price")
    plt.ylabel("USD")
    plt.tight_layout()
    plt.savefig(f'{client_name}_wbp_price.png')
    plt.close()

    client_df = client_df.round(2)
    client_df = client_df.reset_index()
    client_df.Date = client_df.Date.dt.strftime('%B %Y')
    
    max_row, max_col = client_df.shape
    xlfilepath = os.path.join(clients_path, f'{client_name}_wbp_prices.xlsx')
    with pd.ExcelWriter(xlfilepath, engine='xlsxwriter') as writer:
        client_df.to_excel(writer, sheet_name=client_name, startrow=1, header=False, index=False)
        workbook = writer.book
        worksheet = writer.sheets[client_name]
        column_settings = [{'header': column} for column in client_df.columns]
        worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})
        worksheet.insert_image(max_row + 3, 1, f'{client_name}_wbp_price.png')

NameError: name 'os' is not defined